# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BarnesDoubleGamma, BenchmarkTools
import BootstrapVirasoro.LoopModels: compute_reference, ρ_residue
println("Number of threads: ", Threads.nthreads())

Number of threads: 64


# Bootstrap equations

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}
We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [3]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=1 / (big"0.8" + big"0.2" * im))
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
P = big"0.2" + big"0.1" * im
diag_field = Field(c, P=P)
fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))

wrs(r, s, β) = 2cospi(r * β^2 - s)

# Determine the parity of the number of legs in 4pt channels
function chan_parities(co::Correlation4)
    V1, V2, V3, V4 = co.fields
    chan_parities = Channels{Rational}((V1.r + V2.r) % 1, (V1.r + V4.r) % 1, (V1.r + V3.r) % 1)
end

# Compute series of blocks for any channel field compatible
# with the correlation co
function LoopSpectra(co, fields, fs)
    Vs = @channels filter(V -> V.r % 1 == chan_parities(co)[chan], fields)
    @channels ChannelSpectrum(co, chan, Vs[chan], fs[chan])
end

# Compute series of blocks, keeping only one of each (field, reflected field) pair
# given indices of the external fields.
function precompute_blocks(
    indices, fields=nothing;
    c=CC(β=1 / (big"0.8" + big"0.2" * im)),
    P=big"0.2" + big"0.1" * im,
    parity, precision=10
)
    setprecision(BigFloat, floor(Int, 1.2 * precision), base=10)
    Δmax = floor(Int, 1.5 * precision)
    if fields === nothing
        ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
        fields = vcat([Field(c, r=r, s=s) for (r, s) in ndiag_indices], Field(c, P=P))
    end
    co = Correlation([Field(c, r=r, s=s) for (r, s) in indices], Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    fs = Channels(chan -> (V -> IBlock(co, chan, V, parity=parity)))
    LoopSpectra(co, fields, fs)
end

function compute_diagblocks(specs, Ps)
    co = specs.s.corr
    c = co.fields[1].c
    return @channels [IBlock(co, chan, Field(c, P=P)) for P in Ps[chan]]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
    specs, signature;
    even_spin=Channels(false),
    rmax=3, show=true, Ps=nothing,
    fix=nothing,
    rels=nothing,
)
    specs = @channels filter(V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan], specs[chan])
    specs = @channels filter(V -> even_spin[chan] ? spin(V) % 2 == 0 : true, specs[chan])
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix, rels=rels)
        if show
            if rels === :stu
                printstyled("Channels s, t, u:\n", bold=true)
                Base.show(stdout, res.str_cst.s, rmax=rmax)
            else
                Base.show(stdout, res.str_cst, rmax=rmax)
            end
        end
    else
        diagblocks = compute_diagblocks(specs, Ps)
        res = solve_bootstrap(specs, diagblocks, fix=fix, rels=rels)
    end
    return res
end;

# Some numerical four-point functions

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [8]:
ind = ((1//2, 0), (1//2, 0), (1//2, 0), (1//2, 0))
blocks = precompute_blocks(ind, fields, parity=0, precision=20);

### Signature $(0, 1, 1)$

In [9]:
sol = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ P = (0.2+0.1im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ -0.167173+0.134319im      │ 5e-18     │
│ (2, 0)          │ -0.00239852+0.000773346im │ 2.4e-16   │
│ (2, 1)          │ 0.000797319-0.000705013im │ 1.4e-16   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌────────────┬───────────────────────────┬───────────┐
│ Field      │ Structure constant        │ Rel. err. │
├────────────┼───────────────────────────┼───────────┤
│ (1, 0)     │ 0.422102+0.0893706im      │ 1.8e-18   │
│ (1, 1)     │ 0.16568-0.0434621im       │ 8.9e-19   │
│ (2, 0)     │ 0.000999679-1.2823e-05im  │ 4.6e-16   │
│ (2, -1//2) │ 0.000592214+0.000185033im │ 1.1e-15   │
│ (2, 1//2)  │ 0.000592214+0.000185033im │ 9.7e-16   │
│ (2, 1)     │ 0.000344058+0.000230614im │ 6.7e-16   │
└──

In [6]:
# latex output renders in the notebook
display(c)
display(sol.correlation)
# latex copy-pastable output with backend=:latex keyword argument:
show(stdout, sol.str_cst, rmax=2, backend=:latex)

c = 1.614532871972318339100346020761245674740484429065743944636678200692041522491287 + 2.232249134948096885813148788927335640138408304498269896193771626297577854671372im, β = -1.176470588235294117647058823529411764705882352941176470588235294117647058823537 + 0.2941176470588235294117647058823529411764705882352941176470588235294117647058841im

NonChiralCorrelation{Complex{BigFloat}} with external fields
< (1//2, 0) (1//2, 0) (1//2, 0) (1//2, 0) >

 Channel s
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Rel. err.} \\\hline
  $P = (0.2+0.1im)$ & $1.0$ & 0 \\
  $\left(1,0\right)$ & $-0.16717252\;+\;0.13431905\mathrm{i}$ & 4.2e-18 \\
  $\left(2,0\right)$ & $-0.002398521\;+\;0.000773346\mathrm{i}$ & 1.8e-16 \\
  $\left(2,1\right)$ & $0.0007973192\;-\;0.00070501294\mathrm{i}$ & 1.2e-16 \\\hline
\end{tabular}
\\ Channel t
\begin{tabular}{lll}
  \hline
  \textbf{Field} & \textbf{Structure constant} & \textbf{Rel. err.} \\\hline
  $\left(1,0\right)$ & $0.42210183\;+\;0.08937056\mathrm{i}$ & 1.6e-18 \\
  $\left(1,1\right)$ & $0.16567983\;-\;0.04346215\mathrm{i}$ & 6.9e-19 \\
  $\left(2,0\right)$ & $0.0009996791\;-\;1.2822965e-5\mathrm{i}$ & 3.9e-16 \\
  $\left(2,\frac{-1}{2}\right)$ & $0.0005922143\;+\;0.00018503261\mathrm{i}$ & 8.3e-16 \\
  $\left(2,\frac{1}{2}\right)$ & $0.0005922143\;+\;0.00018503261\mathrm{i}$ & 9e-16 \\
  $\left(2,1\right)$ & $0.00034405786\;+\;0.00023061357\mathrm{i}$ & 6e-1

### Signature $(0, 1, 0)$, $D^{(s)}_{(2, 0)} = 0$

In [5]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 1, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                1.0+0.0im │            0.0 │
│          (1, 0) │      -1.16517-0.591022im │    7.65818e-15 │
│          (1, 1) │     0.464566+0.0798147im │     8.4386e-15 │
│          (2, 0) │                0.0+0.0im │            0.0 │
│       (2, 1//2) │ -0.00318409+0.00145081im │    1.01034e-14 │
│      (2, -1//2) │ -0.00318409+0.00145081im │     8.2106e-15 │
│          (2, 1) │  0.00257332-0.00274978im │    6.86828e-15 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │     -0.433576-0.577531im │    1.38597e-14 │
│     (1, 1) │     0.672446+0.0319329i

### Solve for many different Ps

In [78]:
# solve for many different values of the momentum for the diag field in the channel.
# only recompute one column of the linear system each time: very cheap.
Ps = [big"0.8" + big"0.2"*im + i / big(100)*im for i in 1:3];
diagblocks = compute_diagblocks(blocks, Ps, :s)
sol_Ps = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),  Ps=Ps, diagchan=:s)
# print one of the results
show(stdout, sol_Ps[2].str_cst, rmax=2)

InterchiralBlock{Complex{BigFloat}}[G^(s)({ V_{P = 0.80000000000000000000000017 + 0.21000000000000000000000003im + n/β}, n ∈ -5:1:3}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.22000000000000000000000002im + n/β}, n ∈ -5:1:4}), G^(s)({ V_{P = 0.80000000000000000000000017 + 0.23000000000000000000000002im + n/β}, n ∈ -5:1:4})]
 Channel s
┌─────────────────┬─────────────────────────┬───────────┐
│ Field           │ Structure constant      │ Rel. err. │
├─────────────────┼─────────────────────────┼───────────┤
│ (1, 0)          │ 0.253897-0.354829im     │ 5.3e-18   │
│ (P=0.80+0.22im) │ 1.0+0.0im               │ 0         │
│ (2, 0)          │ 0.00488132-0.00418444im │ 2.1e-16   │
│ (2, 1)          │ -0.00118272+0.0024477im │ 1.2e-16   │
└─────────────────┴─────────────────────────┴───────────┘
 Channel t
┌────────────┬─────────────────────────┬───────────┐
│ Field      │ Structure constant      │ Rel. err. │
├────────────┼─────────────────────────┼───────────┤
│ (1, 0)     │ -1.0826

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

## $\left\langle P_1 P_2 P_3 P_4 \right\rangle$

In [7]:
ss = rand(Complex{BigFloat}, 4)
ind = Tuple((0, s) for s in ss)
blocks = precompute_blocks(ind, fields, parity=1, precision=15);

In [8]:
sol = solve(blocks, Sig(0, 0, 0), show=true);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │      -0.043652+0.0461863im │    6.68646e-10 │
│          (1, 1) │     -0.00502903+0.098547im │     1.1386e-10 │
│          (2, 0) │   0.00079105-0.000735744im │     4.28782e-8 │
│       (2, 1//2) │  0.000110591-0.000183974im │     1.46143e-7 │
│          (2, 1) │ -0.000189192-3.69278e-06im │     5.65625e-8 │
│          (3, 0) │                          0 │      0.0607838 │
│       (3, 1//3) │                          0 │      0.0644803 │
│       (3, 2//3) │                          0 │      0.0288652 │
│          (3, 1) │                          0 │      0.0139439 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [9]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks = precompute_blocks(ind, fields2, parity=1, precision=13);

UndefVarError: UndefVarError: `fields2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Signature $(0, \frac32, \frac32)$

In [10]:
sol = solve(blocks, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │              1.0+0.0im │            0.0 │
│          (1, 0) │    -1.79924+0.421555im │      0.0536187 │
│          (1, 1) │ 0.000273845+0.203333im │       0.179268 │
│          (2, 0) │      1.05837-2.37073im │       0.276612 │
│       (2, 1//2) │   -0.557097+0.386062im │        1.79628 │
│          (2, 1) │    -0.133632-0.76917im │       0.431461 │
│          (3, 0) │      41.9787-27.2899im │       0.626559 │
│       (3, 1//3) │     -34.9665+26.8837im │       0.680484 │
│       (3, 2//3) │       24.938-25.2727im │       0.667133 │
│          (3, 1) │     -11.1594+19.0543im │       0.716201 │
└─────────────────┴────────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├─

### Signature $(1, \frac12, \frac32)$

In [11]:
solve(blocks, Sig(1, 1//2, 3//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.314676-0.227065im │        1.58253 │
│    (2, 0) │   -0.756645+5.0774im │       0.595367 │
│ (2, 1//2) │  -0.515941-3.00656im │       0.911785 │
│    (2, 1) │   0.973783+1.64767im │        1.26323 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (1, 0) │ -0.0288501+0.0165005im │          1.011 │
│    (1, 1) │  0.0329273-0.0770832im │       0.777737 │
│    (2, 0) │      3.39492+1.71516im │          2.017 │
│ (2, 1//2) │     -2.78334-1.07935im │        1.75393 │
│    (2, 1) │    1.69856+0.0715347im │        1.71933 │
└───────────┴────────────────────────┴──────

### Signature $(1, \frac32, \frac12)$

In [12]:
solve(blocks, Sig(1, 3//2, 1//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.422351-0.268128im │       0.215619 │
│    (2, 0) │    -2.42269+4.0059im │       0.383119 │
│ (2, 1//2) │    1.60603-2.13824im │       0.575963 │
│    (2, 1) │  -0.166614+1.56393im │       0.488147 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────┬────────────────┐
│ Field     │ Structure constant │ Relative error │
├───────────┼────────────────────┼────────────────┤
│    (2, 0) │  2.02455+1.94047im │       0.673465 │
│ (2, 1//2) │ -1.66763-1.39314im │       0.676533 │
│    (2, 1) │ 1.12007+0.374722im │       0.710549 │
└───────────┴────────────────────┴────────────────┘
 Channel u
┌───────────┬───────────────────────┬────────────────┐
│ Field     │ Structure constant    │ Relative error │
├──────

## $\left\langle (0, \frac12)^3 P_{(1, 0)} \right\rangle$

In [3]:
ind = ((0, 1//2), (0, c.β^2), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
sol_diag = solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 1.35314+0.621346im        │ 3.2e-18   │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 1.9e-16   │
│ (2, 1)          │ -9.09864e-05+0.00247326im │ 1.3e-16   │
│ (3, 0)          │ 1.58764e-06+1.66872e-06im │ 1.9e-12   │
│ (3, 2//3)       │ 1.18639e-06-4.9787e-07im  │ 1.8e-12   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0-8.82188e-20im         │ 3.4e-19   │
│ (1, 0)          │ 1.35314+0.621346im        │ 3e-18     │
│ (2, 0)          │ -0.0146027+0.00422597im   │ 9.6e-17   │
│ (2, 1)          

## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [4]:
ind = ((0, 1//2), (1, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);

In [5]:
sol1 = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),show=true, rmax=2);
sol2 = solve(blocks, Sig(1, 0, 1), even_spin=Channels(false, true, false), show=false);
sol3 = solve(blocks, Sig(1, 1, 0), even_spin=Channels(false, false, true), show=false);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ 0                         │ 0.31      │
│ (2, 0)          │ -0.00585115-0.000436412im │ 1.5e-17   │
│ (2, 1)          │ 0.00227685-0.000981606im  │ 1.5e-17   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌───────────┬──────────────────────────┬───────────┐
│ Field     │ Structure constant       │ Rel. err. │
├───────────┼──────────────────────────┼───────────┤
│ (1, 0)    │ 0.676571+0.310673im      │ 8.6e-20   │
│ (1, 1)    │ 0.286591-0.0446828im     │ 4.8e-20   │
│ (2, 0)    │ -0.00437577+0.00233119im │ 1e-17     │
│ (2, 1//2) │ -0.00327768+0.00228679im │ 9.9e-18   │
│ (2, 1)    │ -0.00118392+0.00172743im │ 8.3e-18   │
└───────────┴──────────────────────────┴───────────┘
 Channel u
┌─────────

In [6]:
show(
    stdout,
    sol1.str_cst.s + sol2.str_cst.s + sol3.str_cst.s - sol_diag.str_cst.s,
    rmax = 2
)

┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.10+0.10im) │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0                          │ 0.31      │
│ (1, 1)          │ 5.17633e-20-1.52054e-19im  │ 6e-19     │
│ (2, 0)          │ -2.60874e-19-4.11794e-18im │ 4.3e-16   │
│ (2, 1//2)       │ 1.18137e-18+1.02885e-18im  │ 2.7e-16   │
│ (2, 1)          │ -6.71868e-19-5.58104e-19im │ 4e-16     │
└─────────────────┴────────────────────────────┴───────────┘


## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [16]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
solve(blocks, Sig(1, 2, 1), even_spin=Channels(false, true, false), rmax=3);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -0.63854+0.181026im │    2.98082e-17 │
│    (2, 0) │                          0 │       0.880898 │
│ (2, 1//2) │                          0 │       0.970237 │
│    (2, 1) │                          0 │        1.06903 │
│    (3, 0) │ -7.86521e-05-0.000322813im │    9.03894e-12 │
│ (3, 1//3) │   4.37238e-05+0.00014278im │    1.86741e-11 │
│ (3, 2//3) │  4.92031e-05+9.64753e-05im │    1.78644e-11 │
│    (3, 1) │ -8.77471e-05-9.14424e-05im │    9.44656e-12 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (2, 0) │    0.413186+0.31

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

# Polynomials

In [6]:
ns = range(-big(0.998), big(0.996), 15)
ws = range(-big(0.998), big(0.996), 6)

βs = [sqrt(acos(-n/2)/π) for n in ns]
cs = [CC(β=β) for β in βs]
Ps = [[acos(w/2)/2/π/c.β for w in ws] for c in cs]

indices = [((0, 1 // 2), (0, β^2), (0, 1 // 2), (0, 1 // 2)) for β in βs]
blocks = [[precompute_blocks(indices[j], c=c, P=P, parity=1, precision=40) for P in Ps[j]] for (j, c) in enumerate(cs)]

sols = [[
    solve(blocks[j][i], Sig(0, 0, 0), even_spin=Channels(true), show=false, rels=:stu)
    for i = eachindex(Ps[1])] for j = eachindex(cs)
];

In [7]:
DGs = Vector(undef, length(ns))
Threads.@threads for i = eachindex(ns)
    DGs[i] = DoubleGamma(cs[i].β)
end

In [57]:
function reduce_to_polyterm(r, s)
    vals = [[sols[j][i].str_cst.s.constants[Field(cs[j], r=r, s=s)] for i = eachindex(sols[j])] for j = eachindex(sols)]
    Threads.@threads for j = 1:length(sols)
        c = cs[j]
        DG = DGs[j]
        V = Field(c, r=r, s=s)
        n = -2cospi(c.β^2)
        Threads.@threads for i = 1:length(sols[j])
            P = Ps[j][i]
            co = sols[j][i].correlation
            diag_field = Field(c, P=P)
            w = 2cospi(2c.β * P)
            DP = compute_reference(co, diag_field, :s, DG)
            DV = compute_reference(co, V, :s, DG)
            vals[j][i] *= DP / DV
            vals[j][i] -= ρ_residue(V, co.fields...) / (w - wrs(V.r, V.s, c.β))
        end
    end
    return vcat([[((2cospi(2cs[j].β * Ps[j][i]), -2cospi(cs[j].β^2)), vals[j][i]) for i = eachindex(sols[j])] for j = eachindex(sols)]...)
end

function fit_polynomial(data, varnames, degs)
    if varnames === (:w, :n) || varnames === (:n, :w)
        to_fit = data
    elseif varnames === (:w,)
        to_fit = [((d[1][1],), d[2]) for d in data[1:length(cs):end]]
    elseif varnames === (:n,)
        to_fit = [((d[1][2],), d[2]) for d in data[1:length(cs)]]
    else
        error("Unsupported varnames")
    end
    BootstrapVirasoro.LoopModels.fit!(Polyfit(varnames, degs), to_fit)
end

fit_polynomial (generic function with 1 method)

## $d_{(r, s)}$ for $(r, s) = (1, 0), (2, 0), (2, 1)$

In [55]:
# only run to reset the data Dict! a bit expensive to recompute.
data = Dict()

Dict{Any, Any}()

In [58]:
for (r, s) in [(1, 0), (2, 0), (2, 1)]
    data[r, s] = reduce_to_polyterm(r, s)
end;

In [83]:
degs_w = [1, 2]
degs_n = [0, 2]

for (r, s) in [(1, 0), (2, 0), (2, 1)]
    display(fit_polynomial(data[r, s], (:w, :n), (degs_w[r], degs_n[r])))
end

(2) * w


(8) *  + (-4) * w + (-4) * w^2 + (-2) * n^2 + (1) * w*n^2 + (1) * w^2*n^2


(8) *  + (4) * w + (-4) * w^2 + (-2) * n^2 + (-1) * w*n^2 + (1) * w^2*n^2


In [60]:
data[3, 0] = reduce_to_polyterm(3, 0);
data[3, 2//3]  = reduce_to_polyterm(3, 2//3);

In [85]:
pf = fit_polynomial([(d[1], 3*d[2]) for d in data[3, 2//3]], (:w, :n), (3, 6))
println(pf)

(-6) * w^3 + (-12) * w*n^2 + (14) * w^3*n^2 + (16) * w*n^4 + (-10) * w^3*n^4 + (-4) * w*n^6 + (2) * w^3*n^6



In [86]:
pf = fit_polynomial([(d[1], 3*d[2]) for d in data[3,0]], (:w, :n), (3, 6))
println(pf)

(32) * w^3*n^2 + (-8) * w*n^4 + (-16) * w^3*n^4 + (2) * w*n^6 + (2) * w^3*n^6



In [87]:
for s in [0, 1//2, 1]
    data[4, s] = reduce_to_polyterm(4, s)
end

In [97]:
for s in 0:1//2:1
    pf = fit_polynomial([(d[1], 2 * d[2]) for d in data[4, s]], (:w, :n), (4, 12))
    println("s = $s")
    show(stdout, pf, cutoff=1e-6)
    println()
end

s = 0//1
(-256) *  + (-128) * w + (128) * w^2 + (896) * n^2 + (256) * w*n^2 + (-256) * w^2*n^2 + (-64) * w^4*n^2 + (-1232) * n^4 + (-72) * w*n^4 + (88) * w^2*n^4 + (176) * w^4*n^4 + (848) * n^6 + (-132) * w*n^6 + (92) * w^2*n^6 + (-172) * w^4*n^6 + (-308) * n^8 + (98) * w*n^8 + (-65) * w^2*n^8 + (73) * w^4*n^8 + (56) * n^10 + (-24) * w*n^10 + (14) * w^2*n^10 + (-14) * w^4*n^10 + (-4) * n^12 + (2) * w*n^12 + (-1) * w^2*n^12 + (1) * w^4*n^12

s = 1//2
(-108) * w^2*n^4 + (36) * w^4*n^4 + (180) * w^2*n^6 + (-60) * w^4*n^6 + (-111) * w^2*n^8 + (37) * w^4*n^8 + (30) * w^2*n^10 + (-10) * w^4*n^10 + (-3) * w^2*n^12 + (1) * w^4*n^12

s = 1//1
(-256) *  + (128) * w + (128) * w^2 + (896) * n^2 + (-256) * w*n^2 + (-256) * w^2*n^2 + (-64) * w^4*n^2 + (-1232) * n^4 + (72) * w*n^4 + (88) * w^2*n^4 + (176) * w^4*n^4 + (848) * n^6 + (132) * w*n^6 + (92) * w^2*n^6 + (-172) * w^4*n^6 + (-308) * n^8 + (-98) * w*n^8 + (-65) * w^2*n^8 + (73) * w^4*n^8 + (56) * n^10 + (24) * w*n^10 + (14) * w^2*n^10 + (-14) 

In [99]:
function chebyshev_T(n, x)
    # Chebyshev polynomial T_n(x) using recurrence
    if n == 0
        return one(x)
    elseif n == 1
        return x
    else
        T_prev2 = one(x)
        T_prev1 = x
        for i in 2:n
            T_curr = 2x * T_prev1 - T_prev2
            T_prev2 = T_prev1
            T_prev1 = T_curr
        end
        return T_prev1
    end
end

mutable struct ChebyshevPolyfit
    varnames::Tuple{Vararg{Symbol}}
    degrees::Tuple{Vararg{Int}}
    nb_monoms::Int
    coeffs::Vector
end

function ChebyshevPolyfit(names, degrees)
    nb_monoms = prod(d + 1 for d in degrees)
    return ChebyshevPolyfit(names, degrees, nb_monoms, [])
end

function fit!(pf::ChebyshevPolyfit, data)
    @assert length(data) > pf.nb_monoms "Need more data points"
    T = float(typeof(data).parameters[1].parameters[2])
    mat = Matrix{T}(undef, length(data), pf.nb_monoms)
    
    for (i, d) in enumerate(data)
        xvec = d[1]
        idx = 1
        for deg_w in 0:pf.degrees[1]
            for deg_n in 0:pf.degrees[2]
                mat[i, idx] = chebyshev_T(deg_w, xvec[1]) * chebyshev_T(deg_n, xvec[2])
                idx += 1
            end
        end
    end
    
    vals = [v for (_, v) in data]
    pf.coeffs = mat \ vals
    return pf
end

fit! (generic function with 1 method)

20-element Vector{Float64}:
 -0.9979999999999998
 -0.8929473684210523
 -0.7878947368421048
 -0.6828421052631581
 -0.5777894736842105
 -0.47273684210526307
 -0.3676842105263159
 -0.26263157894736827
 -0.1575789473684212
 -0.052526315789473595
  0.052526315789473595
  0.1575789473684205
  0.26263157894736827
  0.3676842105263169
  0.4727368421052624
  0.5777894736842115
  0.6828421052631579
  0.7878947368421054
  0.8929473684210532
  0.998

In [32]:
println(data[3, 0])

Tuple{Tuple{Complex{BigFloat}, Complex{BigFloat}}, Complex{BigFloat}}[((-0.99799999999999999822364316059974953532218933105263 - 0.0im, -0.997999999999999998223643160599749535322189331054 - 0.0im), -3.9733575513708601338853240897229239038365521676156 + 7.4812428106463972526324980565693485408859899320372e-29im), ((-0.99799999999999999822364316059974953532218933105263 - 0.0im, -0.89294736842105262998957545948398642634090624357618 - 0.0im), -4.0649044545494080671896757989535752687021552160908 + 7.2418475940620081502708725542588355403937317048449e-29im), ((-0.99799999999999999822364316059974953532218933105263 - 0.0im, -0.78789473684210526175550775836822331735962315609836 - 0.0im), -3.8311160595895836428595519888860225688118273310942 + 6.4863742554846151561345568961094790819004884133129e-29im), ((-0.99799999999999999822364316059974953532218933105263 - 0.0im, -0.68284210526315789352144005725246020837834006861644 - 0.0im), -3.3472405762404580692816059758851989850637836853614 + 5.40524314876476